# COMS-E6998-010: Homework 4 {-}
__Name:__ Nicholas Christman (n2677)
__Due:__ Nov. 22, 2002

In [1]:
from __future__ import print_function, division

import sys
import time
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pprint import pprint

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms

from scipy.optimize import curve_fit
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

# set global seed
seed = 6998
torch.manual_seed(seed)

In [2]:
# use a GPU if there is one available
cuda_availability = torch.cuda.is_available()
if cuda_availability:
    device = torch.device('cuda:{}'.format(torch.cuda.current_device()))
else:
    device = 'cpu'
print('\n*************************')
print('GPU Available: {}'.format(cuda_availability))
print('Current Device: {}'.format(device))
print('*************************\n')
# display the GPU info
if cuda_availability:
    !nvidia-smi


*************************
GPU Available: True
Current Device: cuda:0
*************************

Mon Nov 30 00:31:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    29W /  70W |     10MiB / 15079MiB |      5%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+----------------------------------------------------------------------

## Problem 1 - SSD, ONNX model, Visualization, Inferencing (35) {-}
Following the github repo and ONNX tutorials (links provided below), start with a pretrained Pytorch SSD model and retrain it for your target categories, convert the Pytorch model to ONNX, and then deploy it on ONNX runtime server for inferencing.

For part 1, 2, and 3, refer to the steps in the github repo. For part 4 refer to ONNX tutorial on visualizing and for 5 and 6 refer to ONNX tutorial on inferencing.  

_References:_
* Github repo. Shot MultiBox Detector Implementation in Pytorch.    
Available at https://github.com/qfgaohao/pytorch-ssd
* ONNX tutorial. Visualizing an ONNX Model.   
Available at https://github.com/onnx/tutorials/blob/master/tutorials/VisualizingAModel.md
* ONNX tutorial. Inferencing SSD ONNX model using ONNX Runtime Server.   
Available at https://github.com/onnx/tutorials/blob/master/tutorials/OnnxRuntimeServerSSDModel.ipynb
* Google. Open Images Dataset V5 + Extensions.   
Available at https://storage.googleapis.com/openimages/web/index.html
* The PASCAL Visual Object Classes Challenge 2007.   
Available at http://host.robots.ox.ac.uk/pascal/VOC/voc2007/

### 1.1  Pretrained MobilenetV1 SSD tested locally with  Pascal VOC 2007 dataset (show the test accuracy for the 20 classes)

In [19]:
# getting the trainval data
# ! wget -P data/trainval http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
# ! tar -xf data/trainval/VOCtrainval_06-Nov-2007.tar -C data/trainval

--2020-11-30 06:22:17--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘data/trainval/VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  16.7MB/s    in 28s     

2020-11-30 06:22:45 (15.7 MB/s) - ‘data/trainval/VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]



In [20]:
# getting the  test data 
# ! wget -P data/test http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
# ! tar -xf data/test/VOCtest_06-Nov-2007.tar -C data/test 

--2020-11-30 06:22:54--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘data/test/VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================>] 430.13M  17.2MB/s    in 27s     

2020-11-30 06:23:22 (15.8 MB/s) - ‘data/test/VOCtest_06-Nov-2007.tar’ saved [451020800/451020800]



In [8]:
# getting the model/labels
# ! wget -P pytorch-ssd/models https://storage.googleapis.com/models-hao/mobilenet-v1-ssd-mp-0_675.pth
# ! wget -P pytorch-ssd/models https://storage.googleapis.com/models-hao/voc-model-labels.txt

--2020-11-30 06:09:51--  https://storage.googleapis.com/models-hao/mobilenet-v1-ssd-mp-0_675.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37995286 (36M) [application/octet-stream]
Saving to: ‘pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pth’

mobilenet-v1-ssd-mp 100%[===================>]  36.23M  73.0MB/s    in 0.5s    

2020-11-30 06:09:52 (73.0 MB/s) - ‘pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pth’ saved [37995286/37995286]

--2020-11-30 06:09:52--  https://storage.googleapis.com/models-hao/voc-model-labels.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.135.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 O

In [35]:
# test model with Pascal VOC 2007 dataset
# ! python pytorch-ssd/eval_ssd.py --net mb1-ssd \
# --dataset data/test/VOCdevkit/VOC2007/ \
# --trained_model pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pth \
# --label_file pytorch-ssd/models/voc-model-labels.txt

_Note: this is a snippet from the actual output._

```
pytorch-ssd/eval_ssd.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt_boxes[class_index][image_id] = torch.tensor(all_gt_boxes[class_index][image_id])
It took 0.0654754638671875 seconds to load the model.
process image 0
Load Image: 0.025498 seconds.
Inference time:  0.8003177642822266
Prediction: 0.859447 seconds.
process image 1
Load Image: 0.004779 seconds.
Inference time:  0.007477521896362305
Prediction: 0.059341 seconds.
...
process image 4950
Load Image: 0.004848 seconds.
Inference time:  0.005615711212158203
Prediction: 0.037452 seconds.
process image 4951
Load Image: 0.005250 seconds.
Inference time:  0.005333900451660156
Prediction: 0.039187 seconds.


Average Precision Per-class:
aeroplane: 0.6742489426027927
bicycle: 0.7913672875238116
bird: 0.612096015101108
boat: 0.5616402776942253
bottle: 0.3471256662634949
bus: 0.7742298893362103
car: 0.7284171192326804
cat: 0.8360675520354323
chair: 0.5142295855384792
cow: 0.6244090341627014
diningtable: 0.7060035669312754
dog: 0.7849252606216821
horse: 0.8202146617282785
motorbike: 0.793578272243471
person: 0.7042670984734087
pottedplant: 0.40257147509774405
sheep: 0.6071252282334352
sofa: 0.7549120254763918
train: 0.8270992920206008
tvmonitor: 0.6459903029666852

Average Precision Across All Classes:0.6755259276641954
```

### 1.2  Select two related categories from Google Open Images and finetune the pretrained SSD model. 
* Use _open_images_downloader.py_ script. 
* Use the same parameters that are used in the tutorial.

In [50]:
# getting the Google Open Image data
! python pytorch-ssd/open_images_downloader.py --root data/trainval/open_images \
                                               --class_names "Handgun,Shotgun" \
                                               --num_workers 20

2020-11-30 06:54:26,153 - root - Download https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv.
2020-11-30 06:54:26,189 - root - Download https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv.
2020-11-30 06:54:32,861 - root - Read annotation file data/trainval/open_images/train-annotations-bbox.csv
2020-11-30 06:54:46,268 - root - train bounding boxes size: 1307
2020-11-30 06:54:46,268 - root - Approximate Image Stats: 
2020-11-30 06:54:46,271 - root - Handgun: 561/990 = 0.57.
2020-11-30 06:54:46,271 - root - Shotgun: 429/990 = 0.43.
2020-11-30 06:54:46,271 - root - Label distribution: 
2020-11-30 06:54:46,272 - root - Handgun: 727/1307 = 0.56.
2020-11-30 06:54:46,272 - root - Shotgun: 580/1307 = 0.44.
2020-11-30 06:54:46,272 - root - Shuffle dataset.
2020-11-30 06:54:46,272 - root - Save train data to data/trainval/open_images/sub-train-annotations-bbox.csv.
2020-11-30 06:54:46,284 - root - Download https://storage.googleapis.co

In [53]:
# getting the Open Image models
! wget -P pytorch-ssd/models https://storage.googleapis.com/models-hao/gun_model_2.21.pth
! wget -P pytorch-ssd/models https://storage.googleapis.com/models-hao/open-images-model-labels.txt

--2020-11-30 06:59:05--  https://storage.googleapis.com/models-hao/gun_model_2.21.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27044080 (26M) [application/octet-stream]
Saving to: ‘pytorch-ssd/models/gun_model_2.21.pth’

gun_model_2.21.pth  100%[===================>]  25.79M  64.6MB/s    in 0.4s    

2020-11-30 06:59:06 (64.6 MB/s) - ‘pytorch-ssd/models/gun_model_2.21.pth’ saved [27044080/27044080]

--2020-11-30 06:59:06--  https://storage.googleapis.com/models-hao/open-images-model-labels.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26 [text/plain]
Sa

In [63]:
# finetune the SSD model for the new data
# ! cd pytorch-ssd
! python pytorch-ssd/train_ssd.py --dataset_type open_images \
                      --datasets data/trainval/open_images \
                      --net mb1-ssd \
                      --pretrained_ssd pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pth \
                      --checkpoint_folder pytorch-ssd/models \
                      --scheduler cosine \
                      --lr 0.01 \
                      --t_max 100 \
                      --validation_epochs 5 \
                      --num_epochs 2 \
                      --base_net_lr 0.001 \
                      --batch_size 5

2020-11-30 07:06:51,518 - root - INFO - Use Cuda.
2020-11-30 07:06:51,518 - root - INFO - Namespace(balance_data=False, base_net=None, base_net_lr=0.001, batch_size=5, checkpoint_folder='pytorch-ssd/models', dataset_type='open_images', datasets=['data/trainval/open_images'], debug_steps=100, extra_layers_lr=None, freeze_base_net=False, freeze_net=False, gamma=0.1, lr=0.01, mb2_width_mult=1.0, milestones='80,100', momentum=0.9, net='mb1-ssd', num_epochs=2, num_workers=4, pretrained_ssd='pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pth', resume=None, scheduler='cosine', t_max=100.0, use_cuda=True, validation_dataset=None, validation_epochs=5, weight_decay=0.0005)
2020-11-30 07:06:51,519 - root - INFO - Prepare training datasets.
2020-11-30 07:06:52,101 - root - INFO - Dataset Summary:Number of Images: 961
Minimum Number of Images for a Class: -1
Label Distribution:
	Handgun: 727
	Shotgun: 580
2020-11-30 07:06:52,102 - root - INFO - Stored labels into file pytorch-ssd/models/open-images-m

In [67]:
# convert the models to ONNX format
! python pytorch-ssd/convert_to_caffe2_models.py \
                    mb1-ssd pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pth \
                    pytorch-ssd/models/voc-model-labels.txt 

Traceback (most recent call last):
  File "pytorch-ssd/convert_to_caffe2_models.py", line 47, in <module>
    torch.onnx.export(net, dummy_input, model_path, verbose=False, output_names=['scores', 'boxes'])
  File "/opt/conda/lib/python3.7/site-packages/torch/onnx/__init__.py", line 148, in export
    strip_doc_string, dynamic_axes, keep_initializers_as_inputs)
  File "/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py", line 66, in export
    dynamic_axes=dynamic_axes, keep_initializers_as_inputs=keep_initializers_as_inputs)
  File "/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py", line 416, in _export
    fixed_batch_size=fixed_batch_size)
  File "/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py", line 279, in _model_to_graph
    graph, torch_out = _trace_and_get_graph_from_model(model, args, training)
  File "/opt/conda/lib/python3.7/site-packages/torch/onnx/utils.py", line 236, in _trace_and_get_graph_from_model
    trace_graph, torch_out, inputs_states